In [312]:
import numpy as np  # Fast math and arrays
import matplotlib.pyplot as plt  # Displays

# Variables in neural networks

The problem of using variables can be understood as the process of assigning a _filler_ (or _value_) to a _role_ (or _placeholder_). An classic example is the assignment of proper thematic roles to the nouns in the sentence "The boy kicks the ball".  To understand this sentence, each of the two nouns needs to be assigned to one of the possible thematic roles in a semantic representation. In this case, the representation must contain the fact that "the boy" is the __agent__, and that "the ball" is the __patient__.

There are multiple strategies to do so. 

## Superimposition

In _superimposition_ the two vectors representing the value __v__ and the role __r__ are simply summed, so that the final representation is given by:

$$
\mathbf{R} = \mathbf{v} + \mathbf{r}
$$

A representation can be queried by subtracting the corresponding question. So, if we want to know the role of a vector __v__:

$$
\mathbf{r} = \mathbf{R} - \mathbf{v} 
$$


In [313]:
boy = np.array([[1, 0, 0, 0]])
agent = np.array([[0, 0, 1, 0]])
ball = np.array([[0, 1, 0, 0]])
patient = np.array([[0, 0, 0, 1]])

DICTIONARY = {'boy' : boy, 'agent' : agent, 'ball' : ball, 'patient' : patient}

def bind(value, role):
    return value + role

def query(representation, question):
    return representation - question

def identity(obj1, obj2):
    return np.all(obj1 == obj2)

def verify(representation, role, value):
    return np.all((representation - role - value) >= 0)

We can now perform simple operations

In [314]:
r = bind(boy, agent)
answer = query(r, boy) # Which role is the boy?
print ("What role does the Boy have?\n%s" % answer)
print ("Is the answer 'Agent'? %s" % identity(answer, agent))
print("Is boy the agent? %s" % verify(r, boy, agent))
print("Is the boy the patient? %s" % verify(r, boy, patient))

What role does the Boy have?
[[0 0 1 0]]
Is the answer 'Agent'? True
Is boy the agent? True
Is the boy the patient? False


### Problems with superimposition

Superimposition suffers from illusory conjunction

In [315]:
r1 = bind(boy, agent)
r2 = bind(ball, patient)
answer = query(r1 + r2, boy)
print ("What role does the Boy have?\n%s" % answer)
print("Is boy the agent? %s" % verify(r1 + r2, boy, agent))
print("Is the boy the patient? %s" % verify(r1 + r2, boy, patient))   # ERROR!

What role does the Boy have?
[[0 1 1 1]]
Is boy the agent? True
Is the boy the patient? True


## Conjunctive representation

In conjunctive representations, a variable is bound to its value through the outer product of the corresponding vectors, so  that the final representation __R__ is given by:

$$
\mathbf{R} = \mathbf{v}^T \times \mathbf{r}
$$

A representation can be queried by reversing the operation:

$$
\mathbf{v} = \mathbf{R} \times \mathbf{r}^T
$$


In [316]:
def bind(value, role):
    return value.T.dot(role)

def query(representation, question):
    return representation.dot(question.T).T

def identity(obj1, obj2):
    return np.all(obj1 == obj2)

def verify(representation, role, value):
    return identity(query(representation, role), value)

### Problems with conjunctive representations

Conjunctive representations are powerful, but suffer from the lack of symmetry, i.e., the fact that $\mathbf{v}^T \times \mathbf{r} \neq \mathbf{r}^T \times \mathbf{v}$. This makes it impossible to get consistent answers to the queries "Who is the agent?" and "Which role does the boy have?".  


In [317]:
r1 = bind(boy, agent)
r2 = bind(ball, patient)
answer = query(r1 + r2, agent)
print ("Who is the agent?\n%s" % answer)
print("Is the agent a boy? %s" % verify(r1 + r2, agent, boy))
print("Is the patient the boy ? %s" % verify(r1 + r2, patient, boy))   
print("Is the patient the ball? %s" % verify(r1 + r2, patient, ball))
print("Is the agent the ball? %s" % verify(r1 + r2, agent, ball))
print("\n")
## Errors
print("Is the boy the agent? %s **" % verify(r1 + r2, boy, agent))
print("Is the ball the patient? %s **" % verify(r1 + r2, boy, agent))

Who is the agent?
[[1 0 0 0]]
Is the agent a boy? True
Is the patient the boy ? False
Is the patient the ball? True
Is the agent the ball? False


Is the boy the agent? False **
Is the ball the patient? False **


## Hinton's autoassociator

Hinton's solution is to simply use an autoassociator to store stable role-filler patterns. So, the representation of a value/role binding is given by:

$$
\mathbf{R} = [\mathbf{v}, \mathbf{r}]^T \times [\mathbf{v}, \mathbf{r}]
$$

The representation is then learned by the adjustining the values of the corresponding synaptic matrix:

$$
\mathbf{W} \leftarrow \mathbf{W} + \frac{1}{N} \mathbf{R}
$$



In [318]:
N = 8
x = np.zeros((1, N))
R = np.zeros((N, N))
mask = np.ones((N, N))
np.fill_diagonal(mask, N)

def scalar_binary(input):
    """The binary function (-1, 1)"""
    if input > 0:
        return 1.0
    else:
        return -1.0
    
binary = np.vectorize(scalar_binary)

def scalar_rectify(input):
    return max(0, input)

rectify = np.vectorize(scalar_rectify)

def bind(value, role):
    global R
    pair = np.hstack((binary(value), binary(role)))
    R += pair.T.dot(pair)/float(N)

def update():
    global x, R, mask
    inputs = x.dot(R * mask)
    x = binary(inputs)
    
    
def converge(min_diff = 0.01, max_cycles = 1000):
    i = 1
    previous_x = np.copy(x)
    update()
    diff = np.sum(((x - previous_x) ** 2))
        
    while diff > min_diff and i < max_cycles:
        previous_x = np.copy(x)
        update()
        diff = np.sum(((x - previous_x) ** 2))
        i += 1

                       
def query(representation, role):
    global x
    x = np.hstack((np.zeros((1,4)), role))
    converge()
    return rectify(x[0,0:4])

def identity(obj1, obj2):
    return np.all(obj1 == obj2)

def verify(representation, role, value):
    return identity(query(representation, role), value)

As with conjunctive coding, Hinton networks require specific ways to read the network's response, depending on whether we are asking a question about a value or about a role.

In [319]:
bind(boy, agent)
bind(ball, patient)
query(R, agent)
print ("Who is the agent?\n%s" % answer)
print("Is the agent a boy? %s" % verify(R, agent, boy))
print("Is the patient the boy ? %s" % verify(R, patient, boy))   
print("Is the patient the ball? %s" % verify(R, patient, ball))
print("Is the agent the ball? %s" % verify(R, agent, ball))
print("\n")
## Errors
print("Is the boy the agent? %s **" % verify(r1 + r2, boy, agent))
print("Is the ball the patient? %s **" % verify(r1 + r2, boy, agent))

Who is the agent?
[[1 0 0 0]]
Is the agent a boy? True
Is the patient the boy ? False
Is the patient the ball? True
Is the agent the ball? False


Is the boy the agent? False **
Is the ball the patient? False **


## Holographic reduced representations

Holographic representations use convolution to store value/variable bindings in a new vector. 

Representations store pairs of value/role bindings using __cyclic convolution__: $\mathbf{R} = \mathbf{v} \ast \mathbf{r}$

$$
\mathbf{R_j} = \sum_{i}^{n}\mathbf{v}_{i}\mathbf{r}_{j-i} 
$$

Values can be queries from a representation using the __circular correlation__:

$$
\mathbf{r_j} = \sum_{i}^{n}\mathbf{R}_{i}\mathbf{v}_{j-i} 
$$


In [320]:
def bind(x, y):
    """Circular convolution"""
    n = x.shape[1]
    r = np.zeros(x.shape)
    for j in range(n):
        e = 0.0
        for i in range(n):
            xi = i
            yi = (j - i) % n
            e += (x[0,xi] * y[0,yi])
        r[0,j] = e
    return np.array(r)

def query(x, R):
    """Circular correlation"""
    n = R.shape[1]
    y = np.zeros(R.shape)
    for j in range(n):
        e = 0.0
        for i in range(n):
            xi = i
            Ri = (i + j) % n
            e += (x[0,xi] * R[0,Ri])
        y[0,j] = e
    return np.array(y)

def identity(obj1, obj2):
    return np.all(obj1 == obj2)


Holographic reduced representations are symmetric, so they can be easily queried in every form. 

In [321]:
R = bind(boy, agent)
answer = query(R, agent)
print("Who is the agent?\n%s" % answer)
print("Is the agent a boy? %s" % verify(R, agent, boy))
print("Is the boy the agent? %s" % verify(R, boy, agent))

Who is the agent?
[[ 1.  0.  0.  0.]]
Is the agent a boy? True
Is the boy the agent? True


In [321]:
R = bind(boy, agent)
answer = query(R, agent)
print("Who is the agent?\n%s" % answer)
print("Is the agent a boy? %s" % verify(R, agent, boy))
print("Is the boy the agent? %s" % verify(R, boy, agent))

Who is the agent?
[[ 1.  0.  0.  0.]]
Is the agent a boy? True
Is the boy the agent? True
